In [76]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
data = pd.read_csv('train.csv')
data.dropna(inplace = True)

In [23]:
tokenizer = RegexpTokenizer(r'\w+')
data['tokens'] = data['TEXT'].apply(lambda x: tokenizer.tokenize(x))

pos = []
neg = []
for l in data.truth:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
        
data['Pos']= pos
data['Neg']= neg

In [25]:
data = data[['ID', 'DATE', 'TEXT', 'tokens', 'truth', 'Pos', 'Neg']]
data.head()

,ID,DATE,TEXT,tokens,truth,Pos,Neg
0,subject671,2018-05-03 04:56:17,there pizza place couple door couple kid worki...,"[there, pizza, place, couple, door, couple, ki...",1,1,0
1,subject671,2018-05-03 05:06:23,pour hot water pitcher americano shot pouring ...,"[pour, hot, water, pitcher, americano, shot, p...",1,1,0
2,subject671,2018-05-03 06:04:10,ombre pink drink cool lime base think lot cust...,"[ombre, pink, drink, cool, lime, base, think, ...",1,1,0
3,subject671,2018-05-03 07:08:17,tall americano double sleeve cup,"[tall, americano, double, sleeve, cup]",1,1,0
4,subject671,2018-05-10 04:16:03,print blank receipt paper write closing list c...,"[print, blank, receipt, paper, write, closing,...",1,1,0


In [28]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.10, 
                                         random_state=42)

In [29]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

1885388 words total, with a vocabulary size of 105829
Max sentence length is 5768


In [38]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

207510 words total, with a vocabulary size of 27817
Max sentence length is 1622


In [37]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [39]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [42]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [50]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 300

In [45]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["TEXT"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["TEXT"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 105829 unique tokens.


In [51]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(105830, 300)


In [52]:
test_sequences = tokenizer.texts_to_sequences(data_test["TEXT"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [71]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',f1_m,precision_m, recall_m])
    model.summary()
    return model

In [69]:
label_names = ['Pos', 'Neg']
y_train = data_train[label_names].values
x_train = train_cnn_data
y_tr = y_train

In [83]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, len(list(label_names)))

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 100, 300)     31749000    input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 99, 200)      120200      embedding_12[0][0]               
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 98, 200)      180200      embedding_12[0][0]               
____________________________________________________________________________________________

In [63]:
num_epochs = 10
batch_size = 32

hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 102238 samples, validate on 11360 samples
Epoch 1/10
102238/102238 [==============================] - 31s 307us/step - loss: 0.2023 - acc: 0.9478 - f1_m: 0.9478 - precision_m: 0.9478 - recall_m: 0.9478 - val_loss: 0.1985 - val_acc: 0.9436 - val_f1_m: 0.9436 - val_precision_m: 0.9436 - val_recall_m: 0.9436
Epoch 2/10
102238/102238 [==============================] - 31s 305us/step - loss: 0.1748 - acc: 0.9489 - f1_m: 0.9489 - precision_m: 0.9489 - recall_m: 0.9489 - val_loss: 0.1978 - val_acc: 0.9466 - val_f1_m: 0.9466 - val_precision_m: 0.9466 - val_recall_m: 0.9466
Epoch 3/10
102238/102238 [==============================] - 31s 303us/step - loss: 0.1356 - acc: 0.9568 - f1_m: 0.9568 - precision_m: 0.9568 - recall_m: 0.9568 - val_loss: 0.2225 - val_acc: 0.9460 - val_f1_m: 0.9460 - val_precision_m: 0.9460 - val_recall_m: 0.9460
Epoch 4/10
102238/102238 [==============================] - 31s 306us/step - loss: 0.0944 - acc: 0.9675 - f1_m: 0.9675 - precision_m: 0.9675 - recall_m: 0

In [56]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

y_pred_bool = np.argmax(predictions, axis=1)

print(classification_report(data_test['truth'], y_pred_bool))

12623/12623 [==============================] - 1s 100us/step


## LSTM with CNN

In [81]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(10589, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

model_conv = create_conv_model()
model.fit(x_train, y_tr, validation_split=0.1, epochs = 10)

Train on 102238 samples, validate on 11360 samples
Epoch 1/10
102238/102238 [==============================] - 31s 302us/step - loss: 0.0406 - acc: 0.9873 - f1_m: 0.9873 - precision_m: 0.9873 - recall_m: 0.9873 - val_loss: 0.4945 - val_acc: 0.9361 - val_f1_m: 0.9361 - val_precision_m: 0.9364 - val_recall_m: 0.9358
Epoch 2/10
102238/102238 [==============================] - 31s 301us/step - loss: 0.0338 - acc: 0.9893 - f1_m: 0.9893 - precision_m: 0.9893 - recall_m: 0.9893 - val_loss: 0.6525 - val_acc: 0.9410 - val_f1_m: 0.9410 - val_precision_m: 0.9409 - val_recall_m: 0.9410
Epoch 3/10
102238/102238 [==============================] - 31s 304us/step - loss: 0.0334 - acc: 0.9895 - f1_m: 0.9895 - precision_m: 0.9895 - recall_m: 0.9895 - val_loss: 0.5629 - val_acc: 0.9376 - val_f1_m: 0.9376 - val_precision_m: 0.9377 - val_recall_m: 0.9376
Epoch 4/10
102238/102238 [==============================] - 31s 303us/step - loss: 0.0317 - acc: 0.9901 - f1_m: 0.9901 - precision_m: 0.9901 - recall_m: 0

In [82]:
from sklearn.metrics import classification_report

predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

y_pred_bool = np.argmax(predictions, axis=1)

print(classification_report(data_test['truth'], y_pred_bool))

12623/12623 [==============================] - 1s 62us/step
              precision    recall  f1-score   support

           0       0.67      0.02      0.04     11938
           1       0.05      0.81      0.09       685

    accuracy                           0.06     12623
   macro avg       0.36      0.42      0.06     12623
weighted avg       0.64      0.06      0.04     12623

